In [1]:
# 모듈 import
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [2]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./configs/cascade_rcnn/cascade_rcnn_x101_64x4d_fpn_1x_coco.py')

root='../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'train.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 2021
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/cascade_rcnn_x101_64x4d_fpn_1x_coco_trash'

# for faster r-cnn
#cfg.model.roi_head.bbox_head.num_classes = 10

# for cascade r-cnn
for bbox_head in cfg.model.roi_head.bbox_head:
    bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)

In [3]:
log_config = dict(
    interval=10,
    hooks=[
        dict(type='TextLoggerHook'),
        dict(
            type='WandbLoggerHook',
            init_kwargs=dict(
                project='p-stage_object_detection',
                name='moon'))])

In [4]:
list(cfg.keys())

['model',
 'dataset_type',
 'data_root',
 'img_norm_cfg',
 'train_pipeline',
 'test_pipeline',
 'data',
 'evaluation',
 'optimizer',
 'optimizer_config',
 'lr_config',
 'runner',
 'checkpoint_config',
 'log_config',
 'custom_hooks',
 'dist_params',
 'log_level',
 'load_from',
 'resume_from',
 'workflow',
 'seed',
 'gpu_ids',
 'work_dir']

In [5]:
# for wandb
cfg.log_config = log_config

In [6]:
cfg.model.test_cfg.rpn.nms.type = 'soft_nms'

In [7]:
cfg.evaluation.metric = 'mAP'

In [8]:
list(cfg.model.train_cfg.rpn_proposal.keys())

['nms_pre', 'max_per_img', 'nms', 'min_bbox_size']

In [9]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

In [10]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 4883, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3965  | 1 [Paper]     | 6352  | 2 [Paper pack]  | 897   | 3 [Metal]   | 936   | 4 [Glass]    | 982   |
| 5 [Plastic]       | 2943  | 6 [Styrofoam] | 1263  | 7 [Plastic bag] | 5178  | 8 [Battery] | 159   | 9 [Clothing] | 468   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [11]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

In [12]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=False)